<a href="https://colab.research.google.com/github/hana-dool/Pytorch/blob/master/10.NN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 기본 setting

우선 필요한 도구를 임포트하고, GPU 연산이 가능할 경우에는 GPU 연산을 할 수 있도록 설정해줍니다.

In [ ]:
import torch
import torch.nn as nn 
device = 'cuda' if torch.cuda.is_available() else 'cpu'
torch.manual_seed(777) # 기본 torch 의 random 고정
if device == 'cuda':
    torch.cuda.manual_seed_all(777) # gpu 사용시 random 고정

# Data setting

In [ ]:
X = torch.FloatTensor([[0, 0], [0, 1], [1, 0], [1, 1]]).to(device) # to('gpu') : gpu 에 텐서를 복사본을 생성해 저장
Y = torch.FloatTensor([[0], [1], [1], [0]]).to(device)

# Model(단층 퍼셉트론) construct

1개의 뉴런<br>
활성화 함수는 sigmoid<br>
x input 은 특성이 2개<br>
y output 은 특성이 1개 (0,1 class)<br>

In [ ]:
linear = nn.Linear(2, 1, bias=True)
sigmoid = nn.Sigmoid()
model = nn.Sequential(linear, sigmoid).to(device)

In [ ]:
# 0 또는 1을 예측하는 이진 분류 문제이므로 비용 함수로는 크로스엔트로피 함수를 사용합니다.
# 비용 함수와 옵티마이저 정의
criterion = torch.nn.BCELoss().to(device) # nn.BCEloss() 는 이진분류에서 사용하는 크로스 엔트로피 함수

# GPU CUDA를 사용할 계획이라면 optimizer를 정의하기 전에 미리 해놓아야 한다 (model 에서)
# optimizer는 argument로 model의 parameter를 입력받는다.
#.cuda()를 쓰면 모델의 parameter가 cpu 대신 gpu에 올라가는 것이므로 다른 object가 된다.
# 따라서 optimizer에 model parameter의 위치를 전달한 후 model 에다가 .cuda()를 실행하면, 학습시켜야 할 parameter는 GPU에 올라가 있는데 optimizer는 cpu에 올라간 엉뚱한 parameter 위치를 참조하고 있는 것이 된다.
optimizer = torch.optim.SGD(model.parameters(), lr=1)


In [ ]:
for step in range(100): 
    optimizer.zero_grad()
    hypothesis = model(X)

    # 비용 함수
    cost = criterion(hypothesis, Y)
    cost.backward()
    optimizer.step()

    if step % 10 == 0: # 100번째 에포크마다 비용 출력
        print(step, cost.item())

0 0.7273974418640137
10 0.695116400718689
20 0.6937781572341919
30 0.6933605670928955
40 0.693223774433136
50 0.69317626953125
60 0.693158745765686
70 0.6931519508361816
80 0.693149209022522
90 0.693148136138916


In [ ]:
# with 을 torch.no_grad() 와 같이 적용된다고 하면 업데이팅이 되지 않는다.
with torch.no_grad():
    hypothesis = model(X)
    predicted = (hypothesis > 0.5).float() # 0.5 를 threshold 로 정하자.
    accuracy = (predicted == Y).float().mean() 
    # float() 을 취함으로서 True -> 1 / Fasle -> 0
    # mean() 을 취함으로서, accuracy 를 계산한다.
    print('모델의 출력값(Hypothesis): ', hypothesis.detach().cpu().numpy())
    print('모델의 예측값(Predicted): ', predicted.detach().cpu().numpy())
    print('실제값(Y): ', Y.cpu().numpy())
    print('정확도(Accuracy): ', accuracy.item())

모델의 출력값(Hypothesis):  [[0.5007057 ]
 [0.5002256 ]
 [0.4999959 ]
 [0.49951577]]
모델의 예측값(Predicted):  [[1.]
 [1.]
 [0.]
 [0.]]
실제값(Y):  [[0.]
 [1.]
 [1.]
 [0.]]
정확도(Accuracy):  0.5


# Model(다층 퍼셉트론) construct

In [ ]:
model = nn.Sequential(
          nn.Linear(2, 10, bias=True), # input_layer = 2, hidden_layer1 = 10
          nn.Sigmoid(),
          nn.Linear(10, 10, bias=True), # hidden_layer1 = 10, hidden_layer2 = 10
          nn.Sigmoid(),
          nn.Linear(10, 10, bias=True), # hidden_layer2 = 10, hidden_layer3 = 10
          nn.Sigmoid(),
          nn.Linear(10, 1, bias=True), # hidden_layer3 = 10, output_layer = 1
          nn.Sigmoid()).to(device)

In [ ]:
criterion = torch.nn.BCELoss().to(device)
optimizer = torch.optim.SGD(model.parameters(), lr=1)  # modified learning rate from 0.1 to 1

In [ ]:
for epoch in range(1000):
    optimizer.zero_grad()
    # forward 연산
    hypothesis = model(X)

    # 비용 함수
    cost = criterion(hypothesis, Y)
    cost.backward()
    optimizer.step()

    # 100의 배수에 해당되는 에포크마다 비용을 출력
    if epoch % 100 == 0:
        print('epoch : {0:3d}   cost : {1:.4f}'.format(epoch, cost.item()))


epoch :   0   cost : 0.0074
epoch : 100   cost : 0.0045
epoch : 200   cost : 0.0032
epoch : 300   cost : 0.0024
epoch : 400   cost : 0.0019
epoch : 500   cost : 0.0016
epoch : 600   cost : 0.0014
epoch : 700   cost : 0.0012
epoch : 800   cost : 0.0011
epoch : 900   cost : 0.0009


In [ ]:
with torch.no_grad():
    hypothesis = model(X)
    predicted = (hypothesis > 0.5).float()
    accuracy = (predicted == Y).float().mean()
    print('모델의 출력값(Hypothesis): ', hypothesis.detach().cpu().numpy())
    print('모델의 예측값(Predicted): ', predicted.detach().cpu().numpy())
    print('실제값(Y): ', Y.cpu().numpy())
    print('정확도(Accuracy): ', accuracy.item())

모델의 출력값(Hypothesis):  [[7.059432e-04]
 [9.990746e-01]
 [9.991103e-01]
 [8.872806e-04]]
모델의 예측값(Predicted):  [[0.]
 [1.]
 [1.]
 [0.]]
실제값(Y):  [[0.]
 [1.]
 [1.]
 [0.]]
정확도(Accuracy):  1.0
